**Description**: the following script is dedicated to communicate and control (retrieve info and send commands) Xtender via scom

**Developing Operating System**: Windows 7 Enterprise SP1

**Developing Environment**: Anaconda Python 2.7.13

**First created**: 10/03/2017 (10th March 2017)

**Last modified**: 06/04/2017

**Author**: Minghao Xu

**Scom Version**: 1.6.26

**Xtender Version**: 1.6.22

**BSP Version**: 1.6.14

**Script Version**: 0.15

## 1. Import packages

In [1]:
import serial
import subprocess
import os
import numpy as np
import time
from datetime import datetime
from datetime import timedelta

## 2. Set pre-defined paramets and functions

### 2.1 Pre-define parameters and paths

In [2]:
# paths
cwd  = os.getcwd()
dir_scom = 'H:\Profiles_Do_Not_Delete\campus\Desktop\SoLa Kit\scom.exe '
# port
port_name = 'COM1'
# whether to display full info of fetched info
display_output = False
# whether to chech port
chech_port = True
# whether to test communication protocol with serial port
test_comm = True
# whether to open a serial port
open_port = False
# open Xtender
xtender_open = True
# close Xtender
xtender_close = False
# Xtender initialization
extender_init = True
# RCC initialization
rcc_init = True
# BSP initialization
bsp_init = True

### 2.2 Pre-define functions

In [3]:
# The functions below are for reading information from the system and 
# perform scom command to read system info and get data
#------------------------------------------------------------------------------
def read_info(cmd):
    py2output = subprocess.Popen(dir_scom + '--port=' + port_name + ' ' + cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    str_output = py2output.stdout.readlines()
    if display_output:
        for line in str_output:
            print line
    if str_output[-7]=='response:\r\n':
        raw_data = str_output[-1]
        raw_data = raw_data[5:]
        try:
            data = int(raw_data)
        except ValueError:
            data = raw_data
        return data
    else:
        print 'Fetching Info Failure'
        
# perform scom command to send system command
def send_command(cmd):
    py2output = subprocess.Popen(dir_scom + '--port=' + port_name + ' ' + cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    str_output = py2output.stdout.readlines()
    if display_output:
        for line in str_output:
            print line
    if str_output[-5]!='debug: rx bytes:\r\n':
        print 'Sending Command Failure'
        

## 3. Set and open serial port

### 3.1 Check which port is used

In [4]:
# display ports in use by executing shell commands
if chech_port:
    py2output = subprocess.Popen(['mode'], shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in py2output.stdout.readlines():
        print line
    retval = py2output.wait()



Status for device COM3:

-----------------------

    Baud:            38400

    Parity:          None

    Data Bits:       8

    Stop Bits:       1

    Timeout:         ON

    XON/XOFF:        OFF

    CTS handshaking: OFF

    DSR handshaking: OFF

    DSR sensitivity: OFF

    DTR circuit:     ON

    RTS circuit:     ON





Status for device COM1:

-----------------------

    Baud:            38400

    Parity:          Even

    Data Bits:       8

    Stop Bits:       1

    Timeout:         ON

    XON/XOFF:        OFF

    CTS handshaking: OFF

    DSR handshaking: OFF

    DSR sensitivity: OFF

    DTR circuit:     ON

    RTS circuit:     ON





Status for device CON:

----------------------

    Lines:          300

    Columns:        80

    Keyboard rate:  31

    Keyboard delay: 1

    Code page:      1252





### 3.2 Set port number and open it 

In [5]:
# open serial port
if open_port:
    ser = serial.Serial('COM3', baudrate=38400)
    print(ser.name)         # check which port was really used

### 3.3 Test serial port connection and scom

In [6]:
# test if scom protocol is working with current serial port
if test_comm:
    py2output = subprocess.Popen(dir_scom + 'test', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in py2output.stdout.readlines():
        print line
    retval = py2output.wait()





scan port: COM1 

COM1 opened with success

  inverter addr_id=101 with v_bat=28.88 detected

  no inverter addr_id=102 detected

  no inverter addr_id=103 detected

  no inverter addr_id=104 detected

  no inverter addr_id=105 detected

  no inverter addr_id=106 detected

  no inverter addr_id=107 detected

  no inverter addr_id=108 detected

  no inverter addr_id=109 detected



scan port: COM2 COM3 

COM3 opened with success

  port not working with scom protocol: RESPONSE_TIMEOUT (3)



scan port: COM4 COM5 COM6 COM7 COM8 COM9 COM10 COM11 COM12 COM13 COM14 COM15 COM16 COM17 COM18 COM19 COM20 


### 4 System Initialization

### 4.1. Xtender

In [7]:
if xtender_open:
    # Xtender on
    cmd = '--verbose=3 write_property src_addr=1 dst_addr=101 object_type=2 object_id=1415 property_id=5 format=INT32 value=1'
    send_command(cmd)
elif xtender_close:
    # Xtender off
    cmd = '--verbose=3 write_property src_addr=1 dst_addr=101 object_type=2 object_id=1399 property_id=5 format=INT32 value=0'
    send_command(cmd)

### 4.2. RCC

In [8]:
# Get Xtender Time and Date & Synchronise RCC(System) Time and PC Time
# When a PC is connected to Xcom-232i via serial port, the connected PC is recognised
# by the Xtender as an RCC. For such reason, changing the RCC time according to current
# time on the PC can be achieved.
#----------------------------------------------------------------------------------------------
# The time of the real system(RCC) is the value of seconds since 1/1/1970 00:00:00
reference_datetime = datetime(year=1970,month=1,day=1,hour=0,minute=0,second=0)
#----------------------------------------------------------------------------------------------
# Define functions to get and set time
def get_system_time():
    cmd = '--verbose=3 read_property src_addr=1 dst_addr=501 object_type=2 object_id=5002 property_id=5 format=INT32'
    time_delta_second = read_info(cmd)
    system_datetime = reference_datetime + timedelta(seconds=time_delta_second)
    return system_datetime
def synchronise_time():
    current_datetime = datetime.now()
    system_datetime = get_system_time()
    target_time_delta_second = round((current_datetime - reference_datetime).total_seconds())
    cmd = '--verbose=3 write_property src_addr=1 dst_addr=501 object_type=2 object_id=5002 property_id=5 format=INT32 value=' + str(target_time_delta_second)
    time_delta_second = send_command(cmd)
    system_datetime = get_system_time()
    return system_datetime
#-----------------------------------------------------------------------------------------------
if rcc_init:
    start_time = time.time()
    # Get current date and time from PC
    current_datetime = datetime.now()
    #current_year = current_datetime.year
    #current_month = current_datetime.month
    #current_mday = current_datetime.day
    #current_hour = current_datetime.hour
    #current_min = current_datetime.minute
    #current_sec = current_datetime.second
    print 'Local PC datetime is: ' + str(current_datetime)
    # Get current date and time from system(RCC) in seconds (from reference datetime)
    system_datetime = get_system_time()
    print 'Current system datetime is: ' + str(system_datetime)
    print 'Datetime synchronisation ...'
    current_system_datetime = synchronise_time()
    print 'Current system datetime is: ' + str(current_system_datetime)
    elapsed_time = time.time() - start_time
    print 'RCC datetime synchronisation finished, took ' + str(elapsed_time) + ' seconds'

Local PC datetime is: 2017-04-08 19:15:23.029000
Current system datetime is: 2017-04-08 19:15:23
Datetime synchronisation ...
Current system datetime is: 2017-04-08 19:15:23
RCC datetime synchronisation finished, took 0.514999866486 seconds


### 4.3. BSP

In [9]:
# The recommended operation temperature range for lead acid batteries is 10°C and 35°C (best 20°C +/- 5k).
# Higher temperature will seriously reduce service lifr. Lower temperature reduces the available capacity.
# The absolute maximum temperature is 55°C and should exceed 45°C in service. Refer to the following link for
# further information.
# http://docs-europe.electrocomponents.com/webdocs/04a1/0900766b804a179a.pdf
#--------------------------------------------------------------------------------------------------------------------

In [13]:
# BSP ininitial setting for two Sonnenschein-S12/41 A batteries in series
# Please refer to the following links for futher infotmation about the battery 
# used in the syetm.
# http://uk.rs-online.com/web/p/lead-acid-rechargeable-batteries/6521446/
# http://docs-europe.electrocomponents.com/webdocs/04a1/0900766b804a179a.pdf
# http://www.produktinfo.conrad.com/datenblaetter/250000-274999/251241-da-01-de-AKKU_BLEI_41AH_SOLAR_DRY_S12_41A.pdf
#------------------------------------------------------------------------------------------------------------------------
# Destination address for BSP is 601
# Setting list consits of four parameters, i.e., dst_addr, object_id, format, and value
setting_list = []
#---------------------------------------------------------
# 1. Voltage of the DC system (V) --- 6057
# Only one bit 
# 1: Automatic
# 2: 12 V
# 4: 24 V
# 8: 48 V
# Note: In the technical specification of Xtender serial
# protocol (V1.6.20), 6057 should be of the format 'LONG ENUM',
# however, it is not working with the scom. So 'INT32' was tested
# and used here
setting_list.append([601,6057,'INT32',4])
#----------------------------------------------------------
# 2. Nomonal capacity (Ah@C20) --- 6001
setting_list.append([601,6001,'FLOAT',38])
#----------------------------------------------------------
# 3. Nominal discharge duration (C-rating) --- 6002
setting_list.append([601,6002,'FLOAT',20])
#----------------------------------------------------------
# 4. Nominal shunt current (A) --- 6017
setting_list.append([601,6017,'FLOAT',500])
#-----------------------------------------------------------
# 5. Nominal shunt voltage (mV) --- 6018
setting_list.append([601,6018,'FLOAT',50])
#-----------------------------------------------------------
# 6. Use C20(aka C/20) as reference value (1 for yes, 0 for no) --- 6049
setting_list.append([601,6049,'BOOL',1])
#-----------------------------------------------------------
# 7. Battery current limitation activation --- 6058
# boolean format: 1 for yes, 0 for no
setting_list.append([601,6058,'BOOL',1])
#-----------------------------------------------------------
# 8. Max battery charge current --- 6059
# For lead acid battery, the charging current should be between 10 and 30 percent of the rated capacity. 
# A 10Ah battery at 30 percent charges at about 3A; the percentage can be lower. An 80Ah starter battery
# may charge at 8A. (A 10 percent charge rate is equal to 0.1C.)
# For further information please refer to the link below.
# http://batteryuniversity.com/learn/article/charging_with_a_power_supply
# In our system, the capacity at C20 is 38 and 30% of it is 11.4 A. So we
# set our limitation value to 10A
setting_list.append([601,6059,'FLOAT',10])
#-----------------------------------------------------------
# Start initialization loop if enabled
if bsp_init:
    start_time = time.time()
    for i in range(len(setting_list)):
        tmp_CMD = ('--verbose=3 write_property src_addr=1 dst_addr=' + str(setting_list[i][0]) + 
                   ' object_type=2 object_id=' + str(setting_list[i][1]) + ' property_id=5 format=' + 
                   str(setting_list[i][2]) + ' value=' + str(setting_list[i][3]))
        send_command(tmp_CMD)
    elapsed_time = time.time() - start_time
    print 'BSP initializaiton finished, took ' + str(elapsed_time) + ' seconds'

BSP initializaiton finished, took 1.59300017357 seconds


In [11]:
#py2output = subprocess.Popen(dir_scom + '--port=COM1 --verbose=3 read_property src_addr=1 dst_addr=101 object_type=2 object_id=1526 property_id=5 format=INT32', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#str_output = py2output.stdout.readlines()
#response_output = str_output[-7]
#print response_output=='response:\r\n'
#raw_data = str_output[-1]
#data = int(filter(str.isdigit,raw_data))
#print data
cmd = '--verbose=3 write_property src_addr=1 dst_addr=601 object_type=2 object_id=6057 property_id=5 format=FLOAT value=34'
send_command(cmd)

cmd = '--verbose=3 read_property src_addr=1 dst_addr=601 object_type=2 object_id=6001 property_id=5 format=FLOAT'
data = read_info(cmd)
print data


38
